<a href="https://colab.research.google.com/github/Pyfin5/Machine_Learning/blob/main/Supplier_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab-Notebooks/Reddit_listening/

/content/drive/MyDrive/Colab-Notebooks/Reddit_listening


In [3]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [4]:
!pip install \
    praw \
    pandas \
    numpy \
    python-dotenv \
    datetime \
    IPython \
    llama_index \
    langchain_google_vertexai \
    google-cloud-aiplatform \
    praw \
    textwrap3 \
    langextract \
    asyncpraw

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 119.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 78.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s  0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfull

In [5]:
import praw
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
import datetime as dt
import pandas as pd  # Data manipulation library
import sys
from IPython.display import Markdown, display
import pandas as pd
from dotenv import load_dotenv
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.core.llms import ChatMessage
from google.cloud import aiplatform
from praw.models import MoreComments
import textwrap
import langextract as lx
from google.colab import userdata
import asyncpraw as asyncpraw

In [7]:
load_dotenv() # Load environment variables from .env file

reddit = praw.Reddit(client_id=userdata.get("client_id"),
                     client_secret=userdata.get("client_secret"),
                     user_agent=userdata.get("user_agent"),
                     username=userdata.get("username"),
                     password=userdata.get("password"))


# Subreddit to scrape
subreddit = reddit.subreddit('Learnprogramming')

# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)

Display Name: Learnprogramming


In [8]:
submission = reddit.submission(url = "https://www.reddit.com/r/learnprogramming/comments/vcpqq6/whats_up_with_linux_and_software_developers_if_i/")


In [9]:
print(submission)

vcpqq6


In [16]:
# make sure to extract all operating systems mentioned in the post. Each post might contain mention of multiple operating systems, extract them all.
# if you miss an extraction, you will be spanked.

system_prompt = "Extract company names mentioned in the article. Make sure to provide all instances of company names mentioned in the article."  \
"Each article might contain mention of multiple companies, extract them all including multple instances of the same company."\
"Return each mention of each company as well as the sentiment assoicated with that particular mention."

examples = [
            lx.data.ExampleData(
                text=   "Home Depot stuck by its full-year outlook on Tuesday, even as the company came in slightly shy of Wall Street’s expectations for quarterly earnings and revenue." \
                        "The home improvement retailer reiterated that it expects full-year total sales to grow by 2.8% and comparable sales, which take out the impact of one-time factors" \
                        "like store openings and calendar differences, to rise about 1%."
                        ,
                extractions=[
                    lx.data.Extraction(extraction_class="company", extraction_text="Home Depot", attributes = {'Sentiment':'Positive'})
                ]
            ),
            lx.data.ExampleData(
                text=   "Best Buy is launching a third-party marketplace, as it tries to bulk up the variety of merchandise it offers and reverse slower sales." \
                        "Starting on Tuesday, shoppers who go to Best Buy’s website and app will see products and brands that weren’t available there before, including more tech-related accessories" \
                        "like custom video game controllers and some nontech items including seasonal decor and sports collectibles." \
                        "The company’s online marketplace riffs off those of other retailers, such as Amazon" \
                        "and Walmart, by relying on third-party sellers to stock, sell and ship inventory and taking a cut of their sales in the form of a commission." \
                        "Learn to program on whatever you're familiar with. It's just easier."
                        ,
                extractions=[
                    lx.data.Extraction(extraction_class="company", extraction_text=["Best Buy",'Amazon','Walmart'], attributes = {'Sentiment':'Ambiguious'})
                ],
            ),
            lx.data.ExampleData( #New example which is not a user.
                text=   "After spiraling from crisis to crisis over much of the past seven years, Boeing is stabilizing under CEO Kelly Ortberg’s leadership." \
                        "Ortberg, a longtime aerospace executive and an engineer whom the manufacturer plucked from retirement to fix the problem-addled company last year," \
                        "is set this week to outline significant progress since he took the helm a year ago. Boeing reports quarterly results and gives its outlook on Tuesday." \
                        "So far, investors are liking what they’ve been seeing. Shares of the company are up more than 30% so far this year." \
                        "Wall Street analysts expect the aircraft manufacturer to halve its second-quarter losses from a year ago when it reports." \
                        "Ortberg told investors in May that the manufacturer expects to generate cash in the second half of the year." \
                        "Boeing’s aircraft production has increased, and its airplane deliveries just hit the highest level in 18 months."
                        ,
                extractions=[
                    lx.data.Extraction(extraction_class="company", extraction_text="Boeing", attributes = {'Sentiment':'Positive'})
                ]
            ),
            lx.data.ExampleData( #New example which is not a user.
                text=   "John Deere is warning that tariff costs for the agricultural machinery company could reach a total of $600 million for the fiscal 2025 year." \
                        "The company released its fiscal third-quarter earnings report Thursday, beating on the top and bottom lines but posting significant year-over-year decreases in net income and sales." \
                        "The stock sank roughly 7% in midday trading. The company noted that operating profits for the quarter decreased primarily due to higher tariffs and production costs associated with it." \
                        "Deere’s Director of Investor Relations John Beal said on an earnings call with analysts Thursday that the company took a significant hit in the third quarter due to tariffs." \
                        "Tariff costs in the quarter were approximately $200 million, which brings us to roughly $300 million in tariff expense year-to-date based on tariff rates in effect as of today," \
                        "Beal said. “Our forecast for the pre-tax impact of tariffs in fiscal 2025 is now adjusted to nearly $600 million.”"
                        ,
                 extractions = [
                      lx.data.Extraction(extraction_class="company", extraction_text="John Deere", attributes = {'Sentiment':'Negative'})
                 ]
            )
        ]



In [17]:
def extract_entity(input_text, NER_examples, system_prompt, text_name):

        # Define example data with entities in order of appearance
        examples = NER_examples

        result = lx.extract(
            text_or_documents=input_text,
            prompt_description=system_prompt,
            examples=examples,
            model_id="gemini-2.5-flash",
            api_key=userdata.get('GOOGLE_AI_STUDIO_API_KEY')
        )

        # Display entities with positions
        print(f"Input: {input_text}\n")
        print("Extracted entities:")
        for entity in result.extractions:
            position_info = ""
            if entity.char_interval:
                start, end = entity.char_interval.start_pos, entity.char_interval.end_pos
                position_info = f" (pos: {start}-{end})"
            print(f"• {entity.extraction_class.capitalize()}: {entity.extraction_text}{position_info}")
            print(f"  Attributes: {entity.attributes}") #Add print statement


        print(result)
        # Save and visualize the results
        lx.io.save_annotated_documents([result], output_name="{}.jsonl".format(text_name), output_dir=".")

        # Generate the interactive visualization
        html_content = lx.visualize("{}.jsonl".format(text_name))
        with open("{}.html".format(text_name), "w") as f:
          if hasattr(html_content, 'data'):
            f.write(html_content.data)
          else:
            f.write(html_content)

        print("Interactive visualization saved to {}visualization.html".format(text_name))




In [20]:
input_text = "At a 540,000-square-foot facility in Seymour, Indiana, Guardian Bikes is pulling off a unique challenge: making children’s bikes in America. The company says it produces about 12,000 bikes per week, a manufacturing feat that has become increasingly uncommon in the U.S."\
"“All the way back to World War II era, pretty much every bike sold in the United States was made in the U.S.,” said Guardian Bikes co-founder and CEO Brian Riley. “By the time you get to the ’70s, ’80s, ’90s, it all evaporated.”"\
"Over the last four decades, many American factories have shut down as production moved overseas in search of cheaper labor and larger supplier networks. Between 1997 and 2023 the number of U.S. manufacturing firms and plants dropped by 25% as global trade barriers fell, according to the World Resources Institute."\
"Now, as companies like Apple, IBM and Johnson & Johnson"\
"pledge billions towards U.S. manufacturing and politicians call for reshoring, Guardian offers a glimpse into the possibilities and difficulties of being “Made in America.”"\
"“It wasn’t easy,” said Riley. “It even took kind of getting into losing money for a little bit.” "\
"Guardian Bikes used to rely on a Chinese original equipment manufacturer (OEM) to build its product based on specifications provided by Guardian. But long shipping times and quality concerns prompted a pivot."\
"Starting in 2022, Guardian opened its own plant in the Midwest with the help of $19 million in financing from JPMorgan. Riley said the higher cost of producing domestically is offset by automation, lower inventory costs, and in some cases, tariffs."\
"“With the most recent tariff environment, we’re starting to get either cost parity or in some cases, the domestic parts are cheaper than what you can get out of China,” said Riley. "\
"But building a domestic supply chain from scratch isn’t easy. Many parts, like bicycle chains and reflectors, are no longer made in the U.S. at scale."\
"Experts say finding input suppliers is one of many challenges companies face in bringing manufacturing back to the U.S. Meanwhile, countries like China and Vietnam have become manufacturing powerhouses, investing billions more in factories and training."\
"Even with new tariffs and federal subsidies under the CHIPS Act, some economists remain skeptical of a true U.S. “manufacturing renaissance.”"\
"“Think about how much a pair of sneakers would cost if they were made here in the United States, or a phone or any number of the items that you go to a retail store to purchase. It would be a lot more expensive,” said Colin Grabow, associate director at the Cato Institute’s Herbert A. Stiefel Center for Trade Policy Studies."\
"“So we should all welcome the fact that goods are being produced in the most efficient ways possible, because that lowers prices for us and allows us to raise our standard of living.”"

In [21]:
dist_extract = extract_entity(system_prompt=system_prompt, NER_examples = examples, input_text = input_text, text_name = 'company_extract')

DEBUG:absl:Registered GeminiLanguageModel with patterns ['^gemini'] at priority 10
DEBUG:absl:Registered OllamaLanguageModel with patterns ['^gemma', '^llama', '^mistral', '^mixtral', '^phi', '^qwen', '^deepseek', '^command-r', '^starcoder', '^codellama', '^codegemma', '^tinyllama', '^wizardcoder', '^gpt-oss', '^meta-llama/[Ll]lama', '^google/gemma', '^mistralai/[Mm]istral', '^mistralai/[Mm]ixtral', '^microsoft/phi', '^Qwen/', '^deepseek-ai/', '^bigcode/starcoder', '^codellama/', '^TinyLlama/', '^WizardLM/'] at priority 10
DEBUG:absl:Registered OpenAILanguageModel with patterns ['^gpt-4', '^gpt4\\.', '^gpt-5', '^gpt5\\.'] at priority 10
2025-08-19 20:55:44,280 - langextract.debug - DEBUG - [langextract.inference] CALL: BaseLanguageModel.__init__(self=<GeminiLanguageModel>, constraint=Constraint(co...NONE: 'none'>), kwargs={})
2025-08-19 20:55:44,282 - langextract.debug - DEBUG - [langextract.inference] RETURN: BaseLanguageModel.__init__ -> None (0.0 ms)
2025-08-19 20:55:44,284 - langex

✓ Extraction processing complete



INFO:absl:Finalizing annotation for document ID doc_5a68b9c4.
INFO:absl:Document annotation completed.


✓ Extracted 6 entities (1 unique types)
  • Time: 6.20s
  • Speed: 456 chars/sec
  • Chunks: 3
Input: At a 540,000-square-foot facility in Seymour, Indiana, Guardian Bikes is pulling off a unique challenge: making children’s bikes in America. The company says it produces about 12,000 bikes per week, a manufacturing feat that has become increasingly uncommon in the U.S.“All the way back to World War II era, pretty much every bike sold in the United States was made in the U.S.,” said Guardian Bikes co-founder and CEO Brian Riley. “By the time you get to the ’70s, ’80s, ’90s, it all evaporated.”Over the last four decades, many American factories have shut down as production moved overseas in search of cheaper labor and larger supplier networks. Between 1997 and 2023 the number of U.S. manufacturing firms and plants dropped by 25% as global trade barriers fell, according to the World Resources Institute.Now, as companies like Apple, IBM and Johnson & Johnsonpledge billions towards U.S. man

LangExtract: Saving to company_extract.jsonl: 1 docs [00:00, 15.12 docs/s]

✓ Saved 1 documents to company_extract.jsonl



LangExtract: Loading company_extract.jsonl: 100%|██████████| 4.48k/4.48k [00:00<00:00, 2.17MB/s]

✓ Loaded 1 documents from company_extract.jsonl
Interactive visualization saved to company_extractvisualization.html


In [ ]:
#Previous system prompt
system_prompt_old = "Extract operating system being used by the post author"

#Updated system prompt
system_prompt_new = "Extract operating system mentioned by the post author. Make sure to provide all instances of operating systems mentioned in the post." \
"Each post might contain mention of multiple operating systems, extract them all including multple instances of the same operating system."